<a href="https://www.kaggle.com/code/mr0106/google-tunix-hack?scriptVersionId=289114436" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

C

In [ ]:
# STEP 1: Installation and Environment Setup (Run this first)
# This clears old libraries and installs the specific versions needed for Gemma 3 on TPU.

# 1. Purge cache and remove conflicting versions
!pip cache purge
!pip uninstall -y jax jaxlib libtpu-tpuv3 numpy ml-dtypes orbax-checkpoint flax google-tunix qwix

# 2. Install the optimized TPU stack and competition-specific libraries
!pip install -q "jax[tpu]==0.5.1" -f https://storage.googleapis.com/jax-releases/libtpu_releases.html
!pip install -q "numpy<2.0.0" "ml-dtypes>=0.5.0" "flax==0.12.2" "orbax-checkpoint==0.6.4"
!pip install -q "google-tunix[prod]==0.1.3" git+https://github.com/google/qwix.git
!pip install -q safetensors
print("\n✅ SETUP SUCCESSFUL! Now click the 'RESTART SESSION' button in the toolbar.")

In [ ]:
import pandas as pd
import os, jax, jax.numpy as jnp
import flax.nnx as nnx

# --- 1. وظائف التوليد الذكية (Gemma 3 Optimized) ---

def create_gemma3_mask(tokens):
    """إنشاء قناع رباعي الأبعاد (Batch, Heads, Seq, Seq) لتجنب خطأ Unpack"""
    seq_len = tokens.shape[1]
    # القناع السببي: يمنع الموديل من رؤية المستقبل
    mask = jnp.tril(jnp.ones((seq_len, seq_len)))
    # تمديد الأبعاد لتناسب (Batch=1, Heads=1, Seq, Seq)
    return mask[None, None, :, :].astype(jnp.bool_)

@nnx.jit
def predict_step(model, tokens):
    """خطوة التنبؤ باستخدام JIT لضمان السرعة القصوى على TPU"""
    seq_len = tokens.shape[1]
    positions = jnp.arange(seq_len)[None, :]
    mask = create_gemma3_mask(tokens)
    
    # استدعاء الموديل وتلقي المخرجات (تجاوز خطأ الـ 4 مخرجات)
    outputs = model(tokens, positions=positions, attention_mask=mask, cache=None)
    
    # مخرجات Gemma 3 في Tunix هي Tuple، العنصر الأول هو الـ Logits
    logits = outputs[0] if isinstance(outputs, (list, tuple)) else outputs
    return jnp.argmax(logits[:, -1, :], axis=-1)

def generate_answer(model, tokenizer, prompt, max_new=128):
    """حلقة التوليد: تحويل النص، التنبؤ، ثم فك التشفير"""
    curr_tokens = tokenizer(prompt, return_tensors="jax").input_ids
    
    for _ in range(max_new):
        next_t = predict_step(model, curr_tokens)
        # دمج التوكن الجديد مع الحفاظ على شكل المصفوفة (1, N)
        curr_tokens = jnp.concatenate([curr_tokens, next_t.reshape(1, 1)], axis=1)
        if next_t[0] == tokenizer.eos_token_id:
            break
            
    full_text = tokenizer.decode(curr_tokens[0], skip_special_tokens=True)
    # استخراج جزء الموديل فقط من الـ Prompt
    return full_text.split("model")[-1].strip() if "model" in full_text else full_text

# --- 2. التنفيذ (Inference & Submission) ---

# البحث الديناميكي عن ملف المسابقة
input_file = next((os.path.join(r, f) for r, d, fs in os.walk('/kaggle/input') 
                   if any(f.endswith(x) for x in ['test.csv'])), None)

if not input_file:
    print("⚠️ ملف المسابقة غير موجود! تأكد من ضغط 'Add Input' وإضافة بيانات المسابقة.")
    test_df = pd.DataFrame({'id': [0], 'question': ["Solve: 2x + 5 = 15"]})
else:
    test_df = pd.read_csv(input_file)

print(f"🚀 بدء التوليد على TPU لـ {len(test_df)} سؤال...")
results = []

try:
    for i, row in test_df.iterrows():
        # تنسيق Gemma 3 الرسمي للحوار
        prompt = f"<start_of_turn>user\n{row['question']}\nShow your work step by step.<end_of_turn>\n<start_of_turn>model\n"
        
        with mesh: # استخدام الـ Mesh المعرف مسبقاً لتوزيع العمل على TPU
            ans = generate_answer(model, tokenizer, prompt)
            results.append(ans)
        
        if (i+1) % 5 == 0 or i == 0:
            print(f"✅ تم معالجة {i+1}/{len(test_df)}")

    # حفظ النتيجة النهائية
    submission = pd.DataFrame({'id': test_df['id'], 'answer': results})
    submission.to_csv('submission.csv', index=False)
    print("\n✨ تم إنشاء submission.csv بنجاح! جاهز للتسليم الآن.")

except Exception as e:
    print(f"❌ خطأ تقني: {e}")